In [20]:
from utils import load
from copy import deepcopy
from functools import reduce
from itertools import permutations
from math import floor, ceil
import ast

class Num:
    def __init__(self, value):
        self.value = value
    
    def __repr__(self):
        return str(self.value)

class SnailNumber:  
    def __init__(self, left, right):
        self._number = [left, right]
    
    @classmethod
    def from_str(cls, snail_number):
        return cls.from_list(ast.literal_eval(snail_number))
    
    @classmethod
    def from_list(cls, snail_number):
        def _from_list(snail_number):
            if type(snail_number) == int:
                return Num(snail_number)
            elif all(isinstance(x, int) for x in snail_number):
                return SnailNumber(Num(snail_number[0]), Num(snail_number[1]))
            else:
                return SnailNumber(_from_list(snail_number[0]), _from_list(snail_number[1]))
        return _from_list(snail_number)
    
    @property
    def left(self):
        return self._number[0]

    @property
    def right(self):
        return self._number[1]
    
    def __add__(self, other):
        res = SnailNumber(self, other)
        res.reduce()
        return res
    
    def __getitem__(self, item):
        return self._number[item]
    
    def __setitem__(self, index, value):
        self._number[index] = value
    
    def __repr__(self):
        return str([self.left, self.right])
    
    def _preorder(self):
        lst = []
        def __preorder(nr):
            if isinstance(nr, Num):
                lst.append(nr)
            else:
                __preorder(nr.left)
                __preorder(nr.right)
        __preorder(self.left)
        __preorder(self.right)
        return lst

    def _explode(self):
        preorder = self._preorder()
        def __explode(nr, depth=0, parent=None, side=None):
            if isinstance(nr, Num):
                return
            if depth >= 4 and isinstance(nr.left, Num) and isinstance(nr.right, Num):
                idx_left = preorder.index(nr.left)
                idx_right = preorder.index(nr.right)
                if 0 < idx_left:
                    preorder[idx_left - 1].value += preorder[idx_left].value
                if idx_right < len(preorder) - 1:
                    preorder[idx_right + 1].value += preorder[idx_right].value
                if side == 'left':
                    parent[0] = Num(0)
                else:
                    parent[1] = Num(0)
                return True
            return __explode(nr.left, depth+1, nr, 'left') or __explode(nr.right, depth+1, nr, 'right')
        return __explode(self)
    
    def _split(self):
        def __split(nr, parent=None, side=None):
            if isinstance(nr, Num):
                if nr.value >= 10:
                    new_nr = SnailNumber(Num(floor(nr.value / 2)), Num(ceil(nr.value / 2)))
                    if side == 'left':
                        parent[0] = new_nr
                    else:
                        parent[1] = new_nr
                    return True
                else:
                    return False
            return __split(nr.left, nr, 'left') or __split(nr.right, nr, 'right')
        return __split(self)
    
    def reduce(self):
        while self._explode() or self._split():
            continue
    
    def magnitude(self):
        def _magnitude(nr):
            if isinstance(nr, Num):
                return nr.value
            else:
                return 3*_magnitude(nr.left) + 2*_magnitude(nr.right)
        return _magnitude(self)    

In [24]:
data = load('day18.txt')
data2 = [SnailNumber.from_str(i) for i in data]
reduce(lambda x, y: x + y, data2).magnitude()

3806

In [25]:
data2 = [SnailNumber.from_str(i) for i in data]
magnitudes = []
for c in permutations(data2, 2):
    magnitudes.append((deepcopy(c[0]) + deepcopy(c[1])).magnitude())
max(magnitudes)

4727